In [6]:
# import pandas as pd
import csv
import numpy as np
import os
import copy
from os.path import join as pjoin
from glob import iglob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
#!/usr/bin/env python
from __future__ import division

import argparse
import glob
import os
import random
import signal
import time

import torch

import distributed
from models import data_loader, model_builder
from models.data_loader import load_dataset
from models.model_builder import ExtSummarizer
from models.trainer_ext import build_trainer
from others.logging import logger, init_logger

model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers', 'encoder', 'ff_actv', 'use_interval', 'rnn_size']


In [8]:
root_path = '/data/ksb/'
bert_root_path = pjoin(root_path, 'BertSum/PreSumm')
bert_model_dir = pjoin(bert_root_path, 'models')

data_dir = pjoin(root_path, 'cnn-dailymail/finished_files')

three_data_dir = pjoin(root_path, 'three-mat')
three_data_test = pjoin(three_data_dir, 'test')

#### Loss function 비교  

*Trained Model parameter 필요*


In [21]:
import gensim
print(gensim.__version__)
#> '3.6.0'

from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
#> '3.6.0'

# Download the FastText model
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

3.6.0


In [9]:
class Similarity:
    
    def __init__(self, model, documents, tfidf=None, expo=2.0, threshold=0.0, nonzero_limit=100):
        self.model = model
        self.dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])
        self.similarity_matrix = self.model.similarity_matrix(self.dictionary, tfidf=tfidf, 
                                                              threshold=threshold, exponent=expo, 
                                                              nonzero_limit=nonzero_limit)
    
    def get_similarity(self, src_doc, tgt_doc):
        src_sent = ' '.join(src_doc)
        tgt_sent = ' '.join(tgt_doc)

        src_bow = self.dictionary.doc2bow(simple_preprocess(src_sent))
        tgt_bow = self.dictionary.doc2bow(simple_preprocess(tgt_sent))
        
        sim = softcossim(src_bow, tgt_bow, self.similarity_matrix)
        return sim
        
        

In [10]:
import jsonlines
import json

data_list = []
for data_p in iglob(pjoin(three_data_test, '**.json'), recursive=False):
    
    with open(data_p,'r',encoding='utf-8') as f:
        data = json.load(f)
        data_list.append(data)

### Origin candidate set

In [11]:
from rouge import Rouge
rouge = Rouge()

In [12]:
from transformers import BertTokenizer
tok = BertTokenizer.from_pretrained('bert-base-uncased', verbose=False)

def bert_encode(x, max_len=-1):
    cls_token_id = tok.cls_token_id
    sep_token_id = tok.sep_token_id

    _ids = tok.encode(x, add_special_tokens=False)
    ids = [cls_token_id] # [CLS]
    if max_len > 0:
        ids.extend(_ids[:max_len - 2])
    else:
        ids.extend(_ids[:512 - 2])
    ids.append(sep_token_id) # [SEP], meaning end of sentence
    return ids

def bert_decode(x):
    result = tok.decode(x, skip_special_tokens=True)
    return result

In [40]:
def detect_trigram(src, tgt):
    if len(tgt) < 3 or len(src) < 3:
        return False
    
    tgt_trigrams = [(tgt[i-1],tgt[i],tgt[i+1]) for i in range(1,len(tgt)-1)]
    src_trigrams = [(src[i-1],src[i],src[i+1]) for i in range(1,len(src)-1)]
    
    for src_tri in src_trigrams:
        if src_tri in tgt_trigrams:
            return True ## Detect trigram overlapped with target
        
    return False
    

In [41]:
def detect_4_gram(src, tgt):
    if len(tgt) < 4 or len(src) < 4:
        return False 
    tgt_4_grams = [(tgt[i-2], tgt[i-1],tgt[i],tgt[i+1]) for i in range(2,len(tgt)-1)]
    src_4_grams = [(src[i-2], src[i-1],src[i],src[i+1]) for i in range(2,len(src)-1)]
    
    for src_gram in src_4_grams:
        if src_gram in tgt_4_grams:
            return True ## Detect 4-gram overlapped with target
        
    return False
    

In [42]:
def detect_5_gram(src, tgt):
    if len(tgt) < 5 or len(src) < 5:
        return False
    
    tgt_5_grams = [(tgt[i-2], tgt[i-1],tgt[i],tgt[i+1], tgt[i+2]) for i in range(2,len(tgt)-2)]
    src_5_grams = [(src[i-2], src[i-1],src[i],src[i+1], src[i+2]) for i in range(2,len(src)-2)]
    
    for src_gram in src_5_grams:
        if src_gram in tgt_5_grams:
            return True ## Detect 5-gram overlapped with target
        
    return False

In [43]:
def detect_ngram_list(src, tgt_list, n_gram='trigram'):
    
    if n_gram =='trigram':
        return sum([detect_trigram(src, tgt) for tgt in tgt_list]) > 0
    elif n_gram =='4-gram':
        return sum([detect_4_gram(src, tgt) for tgt in tgt_list]) > 0
    elif n_gram =='5-gram' :
        return sum([detect_4_gram(src, tgt) for tgt in tgt_list])>0

In [44]:
def get_candidate_set(sent_set, reference=None, n_gram='trigram'):
    
    assert n_gram in ['trigram', '4-gram','5-gram', None]
        
    if n_gram == 'trigram':
        detect_ngram = detect_trigram
    elif n_gram == '4-gram':
        detect_ngram = detect_4_gram
    elif n_gram == '5-gram':
        detect_ngram = detect_5_gram
    else:
        detect_ngram = None

        
    
    possible_set_ids = []
    
    for sent_id, sent, sent_txt in sent_set:
        possible_2_sent_idx = []
        
        # number of summary sentences = 2
        for tgt_sent_id, tgt_sent, tgt_sent_txt in sent_set[sent_id+1:]:
            
            # Detect n-gram (default= trigram) 
            if n_gram is not None:
                if not detect_ngram(src=sent, tgt=tgt_sent):
                    possible_2_sent_idx.append(set([sent_id, tgt_sent_id]))
            else:
                possible_2_sent_idx.append(set([sent_id, tgt_sent_id]))
                
        possible_3_sent_idx = copy.deepcopy(possible_2_sent_idx)
        
        # number of summary sentences = 3
        for tgt_sent_id, tgt_sent, tgt_sent_txt in sent_set[sent_id+1:]:
            for poss_sent_ids in possible_2_sent_idx:
                
                poss_sent = [sent_set[ids][1] for ids in poss_sent_ids]
                if n_gram is not None:
                    if not detect_ngram_list(src=tgt_sent, tgt_list=poss_sent, n_gram=n_gram):
                        poss_3_ids = copy.deepcopy(poss_sent_ids)
                        poss_3_ids.add(tgt_sent_id)

                        possible_3_sent_idx.append(poss_3_ids)
                else:
                    poss_3_ids = copy.deepcopy(poss_sent_ids)
                    poss_3_ids.add(tgt_sent_id)

                    possible_3_sent_idx.append(poss_3_ids)
                    

        possible_sent_idx = possible_2_sent_idx + possible_3_sent_idx
        
        for ids in possible_sent_idx:
            if not ids in possible_set_ids:
                possible_set_ids.append(ids)

    return possible_set_ids
        

In [45]:
import pylcs

def compute_txt_redundancy_score(candidate_id):

    cand_num = len(candidate_id)
    
    score = torch.zeros([cand_num], dtype=torch.float64)
        
    def _compute_redundancy(cand):
        redundancy = 0.0
        
        for i, src_sen in enumerate(cand):
            for j, tgt_sen in enumerate(cand[i+1:]):
                if i != j:
                    lcs_val = pylcs.lcs(src_sen, tgt_sen)
                    redundancy += lcs_val 
        
        sents_len = sum([len(s) for sents in cand for s in sents])
        return redundancy / sents_len

    for i in range(cand_num):
        score[i] = np.mean(_compute_redundancy(candidate_id[i]))

    return score

In [46]:
new_data_path = pjoin(three_data_dir,'reconstructed_by_doc_sim.jsonl')

In [51]:
# Origin Candidate
origin_ref_rouges = []
origin_redun_scores = []

# Reconstructed Candidate
refine_ref_rouges = []
refine_redun_scores = []
refine_ngram_ref_rouges = []

In [48]:
def get_rouge(src, tgt, rouge_score='rouge-1', metric='f'):
    sc = rouge.get_scores(' '.join(src), ' '.join(tgt))[0]
    return sc[rouge_score][metric]

In [49]:
def sort_by_sim(cand_set, document, cand_num=20):
    sims_cands_set = []
    for c in cand_set:
        sim_score = similar.get_similarity(document, c)
        sims_cands_set.append((sim_score, c))

    sims_cands_set = sorted(sims_cands_set, key=lambda x: x[0], reverse=True)
    return sims_cands_set[:cand_num]

In [ ]:
stop = 0
surpass = 0
with open(new_data_path, 'w', encoding='utf-8') as f:
    writer = jsonlines.Writer(f)
    
    for data in data_list:
        candidates = data['candidates']
        article = data['article']
        abstract = data["abstract"]


        summaries = [cand[0] for cand in candidates]    
        encoded_cand_set = [[bert_encode(s, 180) for s in cs] for cs in summaries]
        threshold = min([cand[1] for cand in candidates]) 
        
        similar = Similarity(model=fasttext_model300, documents=[' '.join(article)])

        sent_set = []

        for i, encoded_cand in enumerate(encoded_cand_set):
            for j, encoded_sent in enumerate(encoded_cand):
        
                sent_id = sum([len(prev) for prev in encoded_cand_set[:i]])+j
                sent_set.append((sent_id, encoded_sent, candidates[i][0][j]))

        reduced_cand_ids = get_candidate_set(sent_set, n_gram=None)
        reduced_cand_sents = [[sent_set[i][2] for i in ids] for ids in reduced_cand_ids]
        
        reduced_cand_ids_5gram = get_candidate_set(sent_set, n_gram='5-gram')
        reduced_cand_sents_5gram = [[sent_set[i][2] for i in ids] for ids in reduced_cand_ids_5gram]
        
        
        # Drop candidate which has lower score than threshold
        fined_cands_set = sort_by_sim(reduced_cand_sents, article)
        fined_cands_set_5gram = sort_by_sim(reduced_cand_sents_5gram, article)

        ## Redundancy score of Reconstructed Summaries
        refined_cands_set_ = [cs[1] for cs in fined_cands_set]
        refined_cands_set_5gram_ = [cs[1] for cs in fined_cands_set_5gram]
        
        redun_score = compute_txt_redundancy_score(refined_cands_set_)
        redun_score_5gram = compute_txt_redundancy_score(refined_cands_set_5gram_)

        scores = [(cs[0], '\n'.join(cs[1])) for cs in fined_cands_set]
        scores = sorted(scores, key=lambda x: x[0], reverse=True)
        
        scores_5gram = [(cs[0], '\n'.join(cs[1])) for cs in fined_cands_set_5gram]
        scores_5gram = sorted(scores_5gram, key=lambda x: x[0], reverse=True)
        
        reconstructed_candidates = [s[:][-1].split('\n') for s in scores[:20]]
        reconstructed_candidates_5gram = [s[:][-1].split('\n') for s in scores_5gram[:20]]
        
        origin_cand_rouge = [round(get_rouge(abstract, cand[0]),4) for cand in candidates]
        new_cand_rouge = [round(get_rouge(abstract, cand),4) for cand in reconstructed_candidates]
        new_cand_rouge_5gram = [round(get_rouge(abstract, cand),4) for cand in reconstructed_candidates_5gram]

        print("Origin ROUGE : {}".format(origin_cand_rouge))
        print("New Candidate ROUGE : (surpass? {})\n {}".format(max(new_cand_rouge) > max(origin_cand_rouge), new_cand_rouge))
        print("New Candidate(5-gram) ROUGE : (surpass? {})\n {}\n".format(max(new_cand_rouge_5gram) > max(origin_cand_rouge),
                                                                        new_cand_rouge_5gram))
        
        origin_ref_rouges.append(np.mean(origin_cand_rouge))
        refine_ref_rouges.append(np.mean(new_cand_rouge))
        refine_ngram_ref_rouges.append(np.mean(new_cand_rouge_5gram))

        if max(new_cand_rouge_5gram) > max(origin_cand_rouge):
            surpass += 1
        
        new_data = {'article':article, 'candidates':candidates, 'abstract':abstract, 'reconstructed_candidates': reconstructed_candidates,
                   'reconstructed_candidates_5gram': reconstructed_candidates_5gram}
        writer.write(new_data)


Origin ROUGE : [0.4301, 0.463, 0.4444]
New Candidate ROUGE : (surpass? False)
 [0.4255, 0.427, 0.3366, 0.3838, 0.4045, 0.4598, 0.4091, 0.4091, 0.4091, 0.2857, 0.4348, 0.404, 0.4419, 0.4419, 0.4419, 0.2718, 0.4096, 0.4096, 0.4096, 0.2449]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.4255, 0.427, 0.3366, 0.3838, 0.4045, 0.4598, 0.4091, 0.4091, 0.4091, 0.2857, 0.404, 0.4419, 0.4419, 0.4419, 0.4096, 0.4096, 0.4096, 0.5618, 0.5618, 0.5618]

Origin ROUGE : [0.4486, 0.3725, 0.4034]
New Candidate ROUGE : (surpass? True)
 [0.3968, 0.4065, 0.3621, 0.3717, 0.3167, 0.4333, 0.4444, 0.2909, 0.4486, 0.4615, 0.3604, 0.3704, 0.4299, 0.4423, 0.3937, 0.4032, 0.3148, 0.2617, 0.4407, 0.4522]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.3968, 0.4065, 0.3621, 0.3717, 0.3167, 0.4333, 0.4444, 0.2909, 0.4486, 0.4615, 0.3604, 0.3704, 0.4299, 0.4423, 0.3148, 0.2617, 0.4182, 0.4299, 0.3366, 0.2474]

Origin ROUGE : [0.4, 0.386, 0.3308]
New Candidate ROUGE : (surpass? True)
 [0.4, 0.4, 0.3939, 0.4, 0.4, 0.

Origin ROUGE : [0.688, 0.7119, 0.7119]
New Candidate ROUGE : (surpass? False)
 [0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.688, 0.688]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.7119, 0.688, 0.688]

Origin ROUGE : [0.404, 0.4037, 0.3784]
New Candidate ROUGE : (surpass? True)
 [0.4, 0.4242, 0.4242, 0.44, 0.4356, 0.404, 0.4, 0.4554, 0.268, 0.4, 0.44, 0.3366, 0.3, 0.4211, 0.4242, 0.4444, 0.4444, 0.433, 0.4356, 0.3711]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.4, 0.4242, 0.4242, 0.44, 0.4356, 0.404, 0.4, 0.4554, 0.268, 0.4, 0.44, 0.3366, 0.3, 0.4211, 0.4242, 0.4444, 0.4444, 0.433, 0.4356, 0.3711]

Origin ROUGE : [0.3191, 0.3721, 0.48]
New Candidate ROUGE : (surpass? False)
 [0.3624, 0.3833, 0.3802, 0.3089, 0.3448, 0.3091, 0.3063, 0.2657, 0

Origin ROUGE : [0.3853, 0.3361, 0.3273]
New Candidate ROUGE : (surpass? False)
 [0.3387, 0.32, 0.2909, 0.2941, 0.2941, 0.3276, 0.3208, 0.3208, 0.3559, 0.3559, 0.3492, 0.2609, 0.2609, 0.3429, 0.2857, 0.2857, 0.3636, 0.339, 0.283, 0.3529]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.3387, 0.32, 0.2909, 0.2941, 0.2941, 0.3276, 0.3208, 0.3208, 0.3559, 0.3559, 0.3492, 0.2609, 0.2609, 0.3429, 0.2857, 0.2857, 0.3636, 0.339, 0.283, 0.3218]

Origin ROUGE : [0.3289, 0.272, 0.3356]
New Candidate ROUGE : (surpass? True)
 [0.2689, 0.4, 0.3586, 0.3239, 0.3662, 0.3279, 0.3361, 0.3193, 0.3636, 0.2946, 0.2879, 0.3016, 0.2326, 0.2636, 0.2642, 0.2791, 0.3256, 0.2381, 0.2857, 0.2791]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.2689, 0.4, 0.3239, 0.3279, 0.3361, 0.3193, 0.3636, 0.3016, 0.2642, 0.3256, 0.2381, 0.3556, 0.2791, 0.2326, 0.2812, 0.2833, 0.2381, 0.3276, 0.2759, 0.2857]

Origin ROUGE : [0.5641, 0.4404, 0.4691]
New Candidate ROUGE : (surpass? True)
 [0.4554, 0.396, 0.3711, 0.4706, 0.41

Origin ROUGE : [0.5789, 0.4867, 0.5761]
New Candidate ROUGE : (surpass? True)
 [0.4893, 0.4609, 0.4234, 0.4502, 0.4178, 0.4248, 0.4511, 0.3333, 0.4686, 0.3382, 0.6122, 0.4224, 0.5868, 0.5896, 0.5761, 0.5159, 0.3662, 0.4867, 0.4867, 0.4938]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.4893, 0.4234, 0.4502, 0.4178, 0.3333, 0.3382, 0.6122, 0.5761, 0.3662, 0.4867, 0.4867, 0.5106, 0.5106, 0.4891, 0.4891, 0.5299, 0.5067, 0.4369, 0.5232, 0.3602]

Origin ROUGE : [0.4112, 0.5747, 0.3542]
New Candidate ROUGE : (surpass? False)
 [0.2857, 0.3784, 0.3297, 0.3542, 0.3396, 0.3654, 0.3146, 0.3542, 0.2727, 0.3462, 0.4045, 0.3297, 0.4176, 0.4158, 0.381, 0.3953, 0.4045, 0.375, 0.4646, 0.321]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.2857, 0.3784, 0.3297, 0.3542, 0.3396, 0.3654, 0.3146, 0.2727, 0.3462, 0.4045, 0.3297, 0.4176, 0.381, 0.3953, 0.4045, 0.4646, 0.321, 0.321, 0.3762, 0.2895]

Origin ROUGE : [0.5645, 0.5586, 0.4404]
New Candidate ROUGE : (surpass? True)
 [0.4074, 0.45, 0.4, 0.367, 

Origin ROUGE : [0.4298, 0.5474, 0.4286]
New Candidate ROUGE : (surpass? False)
 [0.4833, 0.4833, 0.42, 0.42, 0.4298, 0.4298, 0.3762, 0.3762, 0.375, 0.5088, 0.5088, 0.5088, 0.5088, 0.4463, 0.4463, 0.3396, 0.5091, 0.5091, 0.4706, 0.4706]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.4833, 0.4833, 0.42, 0.42, 0.3762, 0.3762, 0.375, 0.4463, 0.4463, 0.3396, 0.5091, 0.5091, 0.4706, 0.4706, 0.4127, 0.5347, 0.5347, 0.4103, 0.4956, 0.4956]

Origin ROUGE : [0.3158, 0.3, 0.3059]
New Candidate ROUGE : (surpass? False)
 [0.3019, 0.3019, 0.3019, 0.3, 0.3, 0.3, 0.303, 0.303, 0.303, 0.3043, 0.3043, 0.3043, 0.2921, 0.2093, 0.2093, 0.2093, 0.3023, 0.3023, 0.3023, 0.2941]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.3019, 0.3019, 0.3019, 0.3, 0.3, 0.3, 0.303, 0.303, 0.303, 0.3043, 0.3043, 0.3043, 0.2921, 0.2093, 0.2093, 0.2093, 0.3023, 0.3023, 0.3023, 0.3059]

Origin ROUGE : [0.4038, 0.6598, 0.6735]
New Candidate ROUGE : (surpass? True)
 [0.4752, 0.4848, 0.4571, 0.7097, 0.6383, 0.4946, 0.5283,

...

Origin ROUGE : [0.5385, 0.4286, 0.4954]
New Candidate ROUGE : (surpass? True)
 [0.4694, 0.4043, 0.4043, 0.3846, 0.5283, 0.4706, 0.5625, 0.4554, 0.4717, 0.4466, 0.4494, 0.4375, 0.5567, 0.52, 0.5, 0.5, 0.5347, 0.3765, 0.3765, 0.3711]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.4694, 0.4043, 0.4043, 0.3846, 0.5283, 0.4706, 0.5625, 0.4554, 0.4717, 0.4466, 0.4494, 0.4375, 0.5567, 0.52, 0.5, 0.5, 0.5347, 0.3765, 0.3765, 0.3711]

Origin ROUGE : [0.3077, 0.386, 0.3619]
New Candidate ROUGE : (surpass? True)
 [0.3564, 0.4272, 0.404, 0.4314, 0.2917, 0.4082, 0.359, 0.359, 0.3208, 0.3208, 0.3059, 0.3505, 0.3704, 0.3704, 0.2807, 0.2807, 0.3462, 0.3462, 0.4318, 0.3585]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.3564, 0.4272, 0.404, 0.4314, 0.2917, 0.4082, 0.359, 0.359, 0.3208, 0.3208, 0.3059, 0.3505, 0.3704, 0.3704, 0.2807, 0.2807, 0.3462, 0.3462, 0.4318, 0.3585]

Origin ROUGE : [0.5, 0.5926, 0.5862]
New Candidate ROUGE : (surpass? True)
 [0.5913, 0.6018, 0.5862, 0.5893, 0.5841, 0.6, 0.4

Origin ROUGE : [0.4952, 0.2727, 0.5]
New Candidate ROUGE : (surpass? False)
 [0.3158, 0.3186, 0.3178, 0.2353, 0.3208, 0.3214, 0.3363, 0.2316, 0.3333, 0.3238, 0.3396, 0.3368, 0.24, 0.2418, 0.3617, 0.2424, 0.2018, 0.3656, 0.2381, 0.3678]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.3158, 0.3186, 0.3178, 0.2353, 0.3208, 0.3214, 0.3363, 0.2316, 0.3333, 0.3238, 0.3396, 0.3368, 0.24, 0.2418, 0.3617, 0.2424, 0.2381, 0.3678, 0.3455, 0.3604]

Origin ROUGE : [0.2979, 0.3469, 0.3333]
New Candidate ROUGE : (surpass? True)
 [0.1856, 0.2653, 0.2418, 0.2737, 0.1875, 0.268, 0.3469, 0.1042, 0.3579, 0.1798, 0.3297, 0.2667, 0.2651, 0.1538, 0.2759, 0.2178, 0.1039, 0.2308, 0.2268, 0.2056]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.1856, 0.2653, 0.2418, 0.2737, 0.1875, 0.268, 0.3469, 0.1042, 0.3579, 0.1798, 0.3297, 0.2667, 0.2651, 0.1538, 0.2759, 0.2178, 0.1039, 0.2308, 0.2268, 0.2056]

Origin ROUGE : [0.5859, 0.68, 0.5789]
New Candidate ROUGE : (surpass? True)
 [0.5802, 0.5802, 0.5556, 0.5789,

Origin ROUGE : [0.4658, 0.4324, 0.3951]
New Candidate ROUGE : (surpass? True)
 [0.3133, 0.325, 0.3377, 0.3377, 0.4359, 0.4359, 0.3514, 0.3514, 0.3855, 0.3333, 0.2432, 0.2338, 0.5128, 0.5128, 0.4658, 0.3611, 0.3243, 0.4, 0.1892, 0.1972]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.3133, 0.325, 0.3377, 0.3377, 0.4359, 0.4359, 0.3514, 0.3514, 0.3855, 0.3333, 0.2432, 0.2338, 0.5128, 0.5128, 0.4658, 0.3611, 0.3243, 0.4, 0.1892, 0.1972]

Origin ROUGE : [0.3874, 0.3804, 0.4045]
New Candidate ROUGE : (surpass? False)
 [0.3854, 0.3684, 0.4, 0.369, 0.3886, 0.3816, 0.3804, 0.4, 0.3494, 0.3699, 0.36, 0.3488, 0.3295, 0.3874, 0.3103, 0.3771, 0.3294, 0.3704, 0.3837, 0.327]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.3854, 0.3684, 0.4, 0.3886, 0.3816, 0.3804, 0.4, 0.3699, 0.36, 0.3295, 0.3874, 0.3103, 0.3771, 0.3704, 0.327, 0.3919, 0.4024, 0.3816, 0.3908, 0.3057]

Origin ROUGE : [0.3776, 0.35, 0.3966]
New Candidate ROUGE : (surpass? True)
 [0.4029, 0.3776, 0.3876, 0.3852, 0.4, 0.3889, 0.33

Origin ROUGE : [0.3519, 0.293, 0.4148]
New Candidate ROUGE : (surpass? False)
 [0.2443, 0.3053, 0.3279, 0.3279, 0.3415, 0.288, 0.2381, 0.2807, 0.3548, 0.2586, 0.2414, 0.2857, 0.2857, 0.3226, 0.2807, 0.2593, 0.2602, 0.2326, 0.2373, 0.193]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.2443, 0.3053, 0.3279, 0.3279, 0.3415, 0.288, 0.2381, 0.2807, 0.3548, 0.2586, 0.2414, 0.2857, 0.2807, 0.2593, 0.2602, 0.2326, 0.193, 0.3089, 0.2946, 0.3333]

Origin ROUGE : [0.4632, 0.7333, 0.6947]
New Candidate ROUGE : (surpass? False)
 [0.4231, 0.4231, 0.5893, 0.5893, 0.5893, 0.5893, 0.5897, 0.5897, 0.5897, 0.5897, 0.386, 0.3158, 0.3158, 0.7333, 0.7333, 0.7333, 0.7333, 0.7333, 0.7333, 0.7333]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.4231, 0.4231, 0.5893, 0.5893, 0.5893, 0.5893, 0.5897, 0.5897, 0.5897, 0.5897, 0.3158, 0.3158, 0.7333, 0.7333, 0.7333, 0.7333, 0.7333, 0.7333, 0.7333, 0.7333]

Origin ROUGE : [0.4231, 0.3864, 0.3579]
New Candidate ROUGE : (surpass? True)
 [0.3846, 0.383, 0.396, 0.

Origin ROUGE : [0.5745, 0.4699, 0.5591]
New Candidate ROUGE : (surpass? False)
 [0.5628, 0.5628, 0.5628, 0.5482, 0.5482, 0.5482, 0.4526, 0.4526, 0.4362, 0.4362, 0.4356, 0.3955, 0.3771, 0.4149, 0.4699, 0.4699, 0.4699, 0.3978, 0.4649, 0.4649]
New Candidate(5-gram) ROUGE : (surpass? False)
 [0.5628, 0.5628, 0.5628, 0.5482, 0.5482, 0.5482, 0.4526, 0.4526, 0.4362, 0.4362, 0.3955, 0.3771, 0.4699, 0.4699, 0.4699, 0.4649, 0.4649, 0.4649, 0.4649, 0.4649]

Origin ROUGE : [0.303, 0.3404, 0.3061]
New Candidate ROUGE : (surpass? True)
 [0.303, 0.303, 0.303, 0.303, 0.3061, 0.3061, 0.3061, 0.3061, 0.3441, 0.3441, 0.3333, 0.2667, 0.2667, 0.3478, 0.3478, 0.2697, 0.2697, 0.2581, 0.2581, 0.2917]
New Candidate(5-gram) ROUGE : (surpass? True)
 [0.303, 0.303, 0.303, 0.303, 0.3061, 0.3061, 0.3061, 0.3061, 0.3441, 0.3441, 0.3333, 0.2667, 0.3478, 0.3478, 0.2697, 0.2581, 0.2581, 0.2917, 0.2917, 0.3371]

Origin ROUGE : [0.7355, 0.9481, 0.8313]
New Candidate ROUGE : (surpass? False)
 [0.7607, 0.5921, 0.7974, 0.75

In [ ]:
(surpass, len(data_list))

In [ ]:
print("Origin Redundancy score : {}".format(round(np.mean(origin_redun), 4)))
print("Origin ROUGE score between reference and summaries : {}".format(round(np.mean(origin_ref_rouges), 4)))

In [ ]:
print("Origin Redundancy score : {}".format(round(np.mean(refine_redun), 4)))
print("Origin ROUGE score between reference and summaries : {}".format(round(np.mean(refine_ref_rouges), 4)))

In [ ]:

with open(new_data_path, 'w', encoding='utf-8') as f:
    writer = jsonlines.Writer(f)
    
    for data in data_list:
        candidates = data['candidates']
        article = data['article']
        abstract = data["abstract"]


        summaries = [cand[0] for cand in candidates]    
        encoded_cand_set = [[bert_encode(s, 180) for s in cs] for cs in summaries]
        threshold = min([cand[1] for cand in candidates]) 
        
        similar = Similarity(model=fasttext_model300, documents=[' '.join(article)])

        sent_set = []

        for i, encoded_cand in enumerate(encoded_cand_set):
            for j, encoded_sent in enumerate(encoded_cand):

                sent_id = sum([len(prev) for prev in encoded_cand_set[:i]])+j
                sent_set.append((sent_id, encoded_sent))

        reduced_cand_ids = get_candidate_set(sent_set, n_gram='5-gram')
        reduced_cand_sents = [[sent_set[i][1] for i in ids] for ids in reduced_cand_ids]
        reduced_cand_set_dec = [[bert_decode(x) for x in cand] for cand in reduced_cand_sents]


        # Drop candidate which has lower score than threshold
        sims_cands_set = []
        for c in reduced_cand_set_dec:
            sim_score = similar.get_similarity(article, c)

            sims_cands_set.append((sim_score, c))

        sims_cands_set = sorted(sims_cands_set, key=lambda x: x[0], reverse=True)
        fined_cands_set = sims_cands_set[:20]

        ## Redundancy score of Original Candidates
        origin_redun = compute_txt_redundancy_score(summaries)
        origin_scores = []
        for re_sc, (ro_sc, sent) in zip(origin_redun, [(cs[1], '\n'.join(cs[0])) for cs in candidates]):
            origin_scores.append((re_sc.item(), ro_sc, sent))

        origin_scores = sorted(origin_scores, key=lambda x: -x[0]*0.1 +x[1], reverse=True)
        
        # Save Top-1 Score
        origin_redun_scores.append(origin_scores[0][0])
        origin_ref_rouges.append(origin_scores[0][1])

        ## Redundancy score of Reconstructed Summaries
        refined_cands_set_ = [cs[1] for cs in fined_cands_set]
        redun_score = compute_txt_redundancy_score(refined_cands_set_)

        scores = []
        for re_sc, (sim_sc, sent) in zip(redun_score, [(cs[0], '\n'.join(cs[1])) for cs in fined_cands_set]):
            scores.append((re_sc.item(), sim_sc, sent))

        scores = sorted(scores, key=lambda x: x[1], reverse=True)
        
        # Save Top-1 Score
        refine_redun_scores.append(scores[0][0])
        refine_ref_rouges.append(scores[0][1])
                
        ## Redundancy score of Reference Summary
        abstract_scores = compute_txt_redundancy_score([abstract])
        
        reconstructed_candidates = [s[:][-1].split('\n') for s in scores[:20]]
        
        origin_cand_rouge = [round(rouge.get_scores(' '.join(abstract), ' '.join(cand[0]))[0]['rouge-l']['f'],4) for cand in candidates]
        new_cand_rouge = [round(rouge.get_scores(' '.join(abstract), ' '.join(cand)[0]['rouge-l']['f'],4) for cand in reconstructed_candidates]
        
        print("Origin ROUGE : {}".format(origin_cand_rouge))
        print("New Candidate ROUGE : {}\n".format(new_cand_rouge))
        
        new_data = {'article':article, 'candidates':candidates, 'abstract':abstract, 'reconstructed_candidates': reconstructed_candidates}
        writer.write(new_data)
